## **WINDOW FUNCTIONS**

*Window functions perform calculations across a set of rows related to the current row, allowing for complex analytics without collapsing rows into a single output. They provide capabilities for running totals, averages, and other aggregations while maintaining individual row details.*
- Operate on a "window" of rows with a relationship to the current row.
- Keep the row's context, unlike aggregate functions that group into single rows.

### **COMMON TABLE EXPRESSIONS (CTE)** <br> 
*Provides a temporary result set that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement. CTEs can make your queries more readable and modular. They are particularly useful for breaking down complex queries into simpler parts.*<br> <br>

CTEs are defined with the WITH clause. They allow you to name and parameterize a subquery, making your SQL scripts more organized and easier to understand. CTEs can be recursive, allowing them to reference themselves, which is useful for hierarchical data querying like organizational charts or category trees.

In [ ]:
-- BASIC SYNTAX FOR CTEs
WITH
    CTE_Name AS (
        -- CTE query here
    )
SELECT
    *
FROM
    CTE_Name;

**EXAMPLE**<br>
Consider a scenario where you have a table named sales with columns for sale_id, sale_date, employee_id, amount, and you want to perform multiple analyses on this data. <br><br>
Let's say you want to:
1. Calculate the total sales for each employee.<br>
2. From those totals, identify employees who achieved sales above a certain threshold in a specific time frame.

Using CTEs, you can break this task into more manageable parts.

In [ ]:
WITH
    TotalSales AS (
        SELECT
            employee_id,
            SUM(amount) AS total_sales
        FROM
            sales
        WHERE
            sale_date BETWEEN '2023-01-01' AND '2023-12-31'
        GROUP BY
            employee_id
    ),
    HighPerformers AS (
        SELECT
            employee_id
        FROM
            TotalSales
        WHERE
            total_sales > 10000
    )
SELECT
    s.employee_id,
    s.sale_id,
    s.sale_date,
    s.amount
FROM
    sales s
    JOIN HighPerformers hp ON s.employee_id = hp.employee_id
WHERE
    s.sale_date BETWEEN '2023-01-01' AND '2023-12-31';

**TotalSales CTE:** Calculates the total sales for each employee within the specified date range. <br><br>
**HighPerformers CTE:** Identifies employees whose total sales exceed the 10,000 threshold. <br><br>
**Final SELECT:** Retrieves detailed sales data for the high performers identified by the CTEs.

### **OVER CLAUSE** <br>
 *The OVER() clause is a fundamental aspect of SQL that enables the use of window functions to perform calculations across sets of rows relative to the current row. It defines how records are partitioned and ordered before the window function is applied, allowing for sophisticated data analysis without collapsing the dataset.* <br> <br>
**Key Components of the OVER() Clause:**
- **Partitioning:** The PARTITION BY component divides the result set into partitions where the window function operates independently.
- **Ordering:** The ORDER BY component specifies the order of rows within each partition.
- **Frame Specification:** Defines the subset of rows within the partition to which the function is applied.

**Related Window Functions:**
- **Aggregation Functions:** SUM(), AVG(), COUNT(), etc., can be used with OVER() to perform partition-wise aggregations.
- **Ranking Functions:** ROW_NUMBER(), RANK(), DENSE_RANK(), and NTILE(), provide ranking capabilities within partitions.
- **Analytical Functions:** LEAD(), LAG(), FIRST_VALUE(), and LAST_VALUE(), for accessing data from different parts of the partition.

**EXAMPLE**<br>
Calculate the cumulative sales within each sales region while also determining the rank of each sale by amount within the same region.

In [ ]:
SELECT
    sales_region,
    sale_date,
    amount,
    SUM(amount) OVER (
        PARTITION BY
            sales_region
        ORDER BY
            sale_date
    ) AS cumulative_sales,
    RANK() OVER (
        PARTITION BY
            sales_region
        ORDER BY
            amount DESC
    ) AS sales_rank
FROM
    sales_data;

**How the OVER() Clause Operates:**
- **Partitioning:** Data is first partitioned by sales_region, treating each region as a separate subset for calculations.
- **Ordering:** Within each partition, rows are ordered by sale_date for cumulative sales and by amount for ranking.
- **Calculations:** For each row, cumulative sales are calculated up to the current sale_date, and a rank is assigned based on amount.

### **PARTITIONS** <br>
*Partitioning in the context of window functions refers to dividing the result set into partitions or groups based on one or more columns. **Each partition is treated as a separate group for the calculation of the window function.** This approach enables you to perform complex analytical operations, like ranking or aggregating, within each partition independently, without collapsing the rows into a single output row as GROUP BY does.*<br>
- **Table Partitioning:** Splits tables into parts based on key columns, making large tables more manageable and queries faster.
- **Index Partitioning:** Works similarly, dividing indexes for quicker access.
- **Partition Strategies:** Range, list, and hash partitioning are common strategies, chosen based on how the data is accessed.

**EXAMPLE**<br>
Partitioning the sales_data table by sales_region to improve query performance on regional reports:

In [ ]:
CREATE TABLE
    sales_data (
        sale_id INT,
        sale_date DATE,
        sales_region VARCHAR(50),
        amount DECIMAL(10, 2)
    )
PARTITION BY
    LIST (sales_region) (
        PARTITION p_north
        VALUES
            IN ('North'),
            PARTITION p_south
        VALUES
            IN ('South'),
            PARTITION p_east
        VALUES
            IN ('East'),
            PARTITION p_west
        VALUES
            IN ('West')
    );

**EXAMPLE**<br>
Consider a table sales_data with columns for sales_region, sale_date, and amount. To rank sales within each region by the sale amount in descending order, you can use the RANK() window function with partitioning.

In [ ]:
SELECT
    sales_region,
    sale_date,
    amount,
    RANK() OVER (
        PARTITION BY
            sales_region
        ORDER BY
            amount DESC
    ) AS rank
FROM
    sales_data;

**Conceptual breakdown:**
- **Partitioning:** The data is divided into partitions based on sales_region. Each sales_region acts as a separate subset of data.
- **Ordering:** Within each partition, rows are ordered by amount in descending order.
- **Calculation:** The RANK() function is applied to each row within its partition, assigning a rank based on the amount. Sales with the highest amounts receive the lowest ranks (i.e., rank 1).
- **Independence:** Calculations are done independently for each partition. The rank starts anew for each sales_region, allowing for comparison within regions.

### **ROW_NUMBER()** <br>
*Generates a unique number for each row to which the window function is applied, based on the order specified in the ORDER BY clause of the window function.*<br> <br>
**EXAMPLE**<br>
Suppose you have a table sales_data with columns for transaction_id, customer_id, purchase_date, and amount. You want to assign a sequential integer to each sale made by a customer, ordered by the date of the purchase.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    ROW_NUMBER() OVER (
        PARTITION BY
            customer_id
        ORDER BY
            purchase_date ASC
    ) AS sale_sequence
FROM
    sales_data;

### **RANK()** <br>
*Assigns a rank to each row within a partition of a result set, with gaps in the ranking sequence when there are ties.*<br> <br>
**EXAMPLE**<br>
In the sales_data table, you want to rank the sales for each customer based on the transaction amount, with higher amounts receiving a higher rank.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    RANK() OVER (
        PARTITION BY
            customer_id
        ORDER BY
            amount DESC
    ) AS rank
FROM
    sales_data;

### **DENSE_RANK()** <br>
*Similar to RANK(), but without gaps in the rank values when there are ties.*<br> <br>
**EXAMPLE**<br>
If you use DENSE_RANK() in the same scenario as RANK(), each sale will receive a rank without gaps, ensuring a continuous ranking sequence even in the case of tie amounts.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    DENSE_RANK() OVER (
        PARTITION BY
            customer_id
        ORDER BY
            amount DESC
    ) AS dense_rank
FROM
    sales_data;

### **NTILE()** <br>
*Distributes the rows in an ordered partition into a specified number of groups and assigns a group number to each row.*<br> <br>
**EXAMPLE**<br>
You want to divide the sales for each customer into quartiles based on the transaction amount.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    NTILE (4) OVER (
        PARTITION BY
            customer_id
        ORDER BY
            amount DESC
    ) AS quartile
FROM
    sales_data;

### **LEAD()** <br>
*Accesses data from a subsequent row in the same result set without the use of a self-join.*<br> <br>
**EXAMPLE**<br>
You want to compare each sale to the next sale made by the same customer in terms of the amount to see if the purchase amount is increasing or decreasing.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    LEAD (amount) OVER (
        PARTITION BY
            customer_id
        ORDER BY
            purchase_date
    ) AS next_sale_amount
FROM
    sales_data;

### **LAG()** <br>
*Accesses data from a previous row in the same result set without the use of a self-join.*<br> <br>
**EXAMPLE**<br>
You're interested in seeing how the current sale's amount compares to the previous sale's amount for the same customer.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    LAG (amount) OVER (
        PARTITION BY
            customer_id
        ORDER BY
            purchase_date
    ) AS prev_sale_amount
FROM
    sales_data;

### **FIRST_VALUE()** <br>
*Returns the first value in an ordered set of values in the window frame.*<br> <br>
**EXAMPLE**<br>
You want to compare each transaction's amount against the first transaction amount for each customer to evaluate growth or decline over time.

In [ ]:
SELECT
    customer_id,
    purchase_date,
    amount,
    FIRST_VALUE (amount) OVER (
        PARTITION BY
            customer_id
        ORDER BY
            purchase_date
    ) AS first_sale_amount
FROM
    sales_data;

### **RECURSIVE COMMON TABLE EXPRESSIONS (CTE)** <br>
*Recursive CTEs are a powerful feature in SQL that allow for the execution of complex queries, such as hierarchical data retrieval, graph traversal, and generating sequences. They extend the functionality of standard CTEs by allowing a CTE to reference itself.*<br><br>
**Key Concepts:**
- **Base Case:** The initial query that returns the CTE's starting point.
- **Recursive Case:** The part of the CTE that references itself to perform iterative calculations or traversals.

**EXAMPLE**<br>
To retrieve a hierarchical list of employees and their managers from an employees table with employee_id and manager_id columns:

In [ ]:
WITH RECURSIVE
    EmployeeHierarchy AS (
        SELECT
            employee_id,
            manager_id,
            1 AS depth
        FROM
            employees
        WHERE
            manager_id IS NULL
        UNION ALL
        SELECT
            e.employee_id,
            e.manager_id,
            eh.depth + 1
        FROM
            employees e
            INNER JOIN EmployeeHierarchy eh ON e.manager_id = eh.employee_id
    )
SELECT
    *
FROM
    EmployeeHierarchy;

Recursive CTEs efficiently handle tasks requiring iteration over rows, such as building hierarchies, without the need for procedural code, simplifying complex SQL queries and enhancing data analysis capabilities.

## **COMPLEX JOINS & SUB-QUERIES** <br>

### **SUB-QUERIES** <br>
Sub-queries, also known as inner queries or nested queries, are SQL queries placed inside another SQL query. They provide a way to break down complex queries into simpler parts, allowing you to isolate specific operations or calculations. Sub-queries can be used in various clauses of an SQL statement, including SELECT, FROM, WHERE, and HAVING.<br><br>
They can be classified based on their position within the main query: <br><br>
**Scalar** sub-queries (return a single value)<br>
**Row** sub-queries (return a single row)<br>
**Column** sub-queries (return a single column)<br>
**Table** sub-queries (return multiple rows and columns).

In [ ]:
-- BASIC SYNTAX FOR SUB-QUERIES
SELECT
    column_names
FROM
    table_name
WHERE
    column_name OPERATOR (
        SELECT
            column_name
        FROM
            table_name
        WHERE
            condition
    );

**EXAMPLE**<br>
Imagine you have the same sales table with columns for sale_id, sale_date, employee_id, and amount. You are tasked with identifying sales transactions that are above the average sales amount. You can achieve this by using a sub-query to first calculate the average sales amount and then use it in the WHERE clause of the main query to filter out the sales transactions.

In [ ]:
SELECT
    sale_id,
    sale_date,
    employee_id,
    amount
FROM
    sales
WHERE
    amount > (
        SELECT
            AVG(amount)
        FROM
            sales
    );

The sub-query (SELECT AVG(amount) FROM sales) calculates the average amount of all sales.<br>
The main query then selects all sales that have an amount greater than this average.

### **COMPLEX JOINS**<br>
**Advanced Join Techniques:**
- **Self-Joins:** Join a table to itself to compare rows within the same table.
- **Non-Equi Joins:** Use conditions other than equality in a JOIN clause, often with comparative operators (<, >, !=).

**SELF-JOIN EXAMPLE**<br>
Find pairs of employees who work in the same department.

In [ ]:
SELECT
    A.employee_name AS Employee1,
    B.employee_name AS Employee2,
    A.department
FROM
    employees A,
    employees B
WHERE
    A.department = B.department
    AND A.employee_id != B.employee_id;

**NON-EQUI-JOIN EXAMPLE**<br>
Assign mentors to employees based on experience, where mentors have more experience than their mentees.

In [ ]:
SELECT
    mentee.name AS Mentee,
    mentor.name AS Mentor
FROM
    employees mentee
    JOIN employees mentor ON mentee.experience_years < mentor.experience_years;

## **DATA TYPE CONVERSION + CASTING** <br>

Data type conversion and casting are pivotal in SQL for transforming data from one type to another, ensuring compatibility between different parts of a query or when performing operations on columns with differing data types. This process can be explicit, where you manually specify the type conversion using functions or keywords, or implicit, where the database management system automatically converts data types as needed.<br><br>
**CAST():** Converts a value from one data type to another specified by the user.<br>
**CONVERT():** Similar to CAST(), but specific to certain SQL dialects (e.g., T-SQL for SQL Server) and may offer more flexibility in some cases.<br>
**Implicit Conversion:** Automatic conversion performed by the SQL engine, depending on the context of the operation<br>

In [ ]:
-- BASIC SYNTAX FOR CAST
CAST(expression AS data_type)

-- BASIC SYNTAX FOR CONVERT
CONVERT(data_type, expression, style)

**EXAMPLE**<br>
Consider a table named sales_log with columns sale_id (integer), sale_date (date), employee_id (integer), and sale_amount (varchar). You need to sum the sale_amount for a report, but sale_amount is stored as a text type instead of a numeric type.<BR><BR>
**Using CAST() to Convert Data Types:**
To perform a sum operation on the sale_amount column, you first need to cast the varchar data to a numeric type.

In [ ]:
SELECT
    SUM(CAST(sale_amount AS DECIMAL(10, 2))) AS total_sales
FROM
    sales_log;

**Using CONVERT() for Date Formatting:** Assuming you want to display the sale_date in a specific format, such as 'YYYY-MM-DD'.

In [ ]:
SELECT
    sale_id,
    CONVERT(varchar, sale_date, 23) AS formatted_sale_date,
    employee_id,
    sale_amount
FROM
    sales_log;

**General Type Conversion:** CAST() is perfectly suited for general-purpose type conversion when no additional formatting is required. It's your go-to for ensuring compatibility across database systems due to its adherence to the SQL standard.<br><br>
**Specific Formatting Needs:** CONVERT() is preferable when working with data that requires specific formatting, particularly date and time data, and when working within database systems that support the CONVERT() function with its additional capabilities.

## **CONDITIONAL LOGIC** <br> 

Conditional logic in SQL allows for decision-making processes within queries, enabling dynamic results based on specific conditions.<br><br>
**Key Functions and Statements:**
- **CASE:** The most versatile conditional logic statement, allowing for multiple conditions and results.
- **IF():** Available in some SQL dialects like MySQL, it evaluates a condition and returns a value if the condition is true, and another if false.
- **IIF():** A shorthand form of CASE available in SQL Server, evaluates a condition, returning one value if true, and another if false.
- **COALESCE:** Returns the first non-null value in a list of arguments. It's useful for handling null values in data analysis.

In [ ]:
-- BASIC SYNTAX FOR CASE STATEMENTS
CASE
    WHEN condition1 THEN result1
    WHEN condition2 THEN result2
    ELSE default_result
END

**EXAMPLE**<br>
Suppose you have a table customers with columns customer_id, name, and total_purchases. You want to categorize customers based on their total_purchases into 'Low', 'Medium', and 'High' segments.

In [ ]:
SELECT customer_id, name, total_purchases,
       CASE
           WHEN total_purchases < 100 THEN 'Low'
           WHEN total_purchases BETWEEN 100 AND 500 THEN 'Medium'
           ELSE 'High'
       END AS purchase_segment
FROM customers;

Customers with total_purchases less than 100 are categorized as 'Low'.<br>
Those with purchases between 100 and 500 are 'Medium'.<br>
All other customers are categorized as 'High'.<br>

## **STRING FUNCTIONS** <br>

**Commonly Used String Functions:**
- **CONCAT():** Combines two or more strings into one.
- **LENGTH():** Returns the length of a string.
- **SUBSTRING():** Extracts a substring from a string.
- **UPPER(), LOWER():** Converts a string to uppercase or lowercase.
- **TRIM():** Removes spaces from the beginning and end of a string.
- **REPLACE():** Replaces occurrences of a specified substring within a string with another substring.

**EXAMPLE**<br>
Given a table customers with a column full_name, you want to standardize the format by trimming leading/trailing spaces, converting names to lowercase, and then capitalizing the first letter of each name.

In [ ]:
SELECT
    CONCAT (
        UPPER(SUBSTRING(TRIM(lower(full_name)), 1, 1)),
        SUBSTRING(TRIM(lower(full_name)), 2)
    ) AS formatted_name
FROM
    customers;

## **DATE + TIME FUNCTIONS** <br>

Date and time functions in SQL handle temporal data, allowing for the manipulation, extraction, and calculation of dates and times. These functions are crucial for querying based on time intervals, formatting date/time values, and performing time-based calculations.<br><br>
**Key Date and Time Functions:**
- **NOW(), SYSDATE:** Returns the current date and time.
- **CURDATE(), CURRENT_DATE():** Return the current date.
- **CURTIME(), CURRENT_TIME():** Return the current time.
- **DATE():** Extracts the date part from a datetime expression.
- **TIME():** Extracts the time part from a datetime expression.
- **DATEDIFF():** Calculates the difference between two dates.
- **DATE_ADD(), DATE_SUB():** Add or subtract a specified time interval from a date.
- **TRUNC():** Truncates a date or number to a specified unit or number of decimal places. In the context of dates, it can reset the time part of a datetime value to midnight or adjust the date to the start of the period specified (e.g., the first day of the month or year).

**EXAMPLE**<br>
For a table subscriptions with start_date and end_date, calculate the duration of each subscription in days.

In [ ]:
SELECT
    DATEDIFF (end_date, start_date) AS duration_days
FROM
    subscriptions;

*For Oracle*, using **SYSDATE** to find records created today:

In [ ]:
SELECT
    *
FROM
    your_table
WHERE
    your_date_column >= TRUNC (SYSDATE)
    AND your_date_column < TRUNC (SYSDATE) + 1;

## **AGGREGATION METHODS** <br>

### **BASIC AGGREGATION METHODS** <br>
Basic aggregation methods in SQL provide essential functionalities for summarizing and analyzing data sets. These methods are pivotal for performing calculations across a range of data points within a single query.<br><br>
**Core Basic Aggregation Functions:**
- **COUNT():** Counts the number of items in a group.
- **SUM():** Adds together all values in a group.
- **AVG():** Calculates the average of values in a group.
- **MAX():** Identifies the maximum value in a group.
- **MIN():** Identifies the minimum value in a group.

**EXAMPLE**<br>
To calculate total sales, average sale amount, and identify the highest and lowest sale amounts from a sales table:

In [ ]:
SELECT
    COUNT(*) AS total_sales,
    SUM(amount) AS total_revenue,
    AVG(amount) AS average_sale_amount,
    MAX(amount) AS highest_sale,
    MIN(amount) AS lowest_sale
FROM
    sales;

### **ADVANCED AGGREGATION METHODS** <br>
Advanced aggregation methods in SQL extend beyond basic functions like SUM() and COUNT(), enabling more complex data summarization. These methods are essential for detailed analytical tasks and reporting.<br><br>
**Key Advanced Aggregation Functions:**
- **GROUP_CONCAT():** In MySQL, concatenates values from a group into a single string with various options for separators.
- **STRING_AGG():** In SQL Server and PostgreSQL, aggregates string values from a group into a single concatenated string.
- **ARRAY_AGG():** In PostgreSQL, aggregates values into an array.
- **JSON_AGG():** Aggregates record values as JSON objects.
- **LISTAGG():** In Oracle, aggregates values from a group into a single string with custom separators.

**EXAMPLE**<br>
Given a table customer_interests with columns customer_id and interest, create a summary of interests per customer as a single string.

In [ ]:
SELECT
    customer_id,
    STRING_AGG (interest, ', ') AS interests
FROM
    customer_interests
GROUP BY
    customer_id;

### **GROUP BY**<br>
*Used to aggregate rows that have the same values in specified columns into summary rows. It precedes the calculation of aggregate functions (e.g., SUM(), AVG()).*<br>

**EXAMPLE**<br>
Aggregate sales by region.

In [ ]:
SELECT
    sales_region,
    SUM(amount) AS total_sales
FROM
    sales_data
GROUP BY
    sales_region;

### **HAVING**<br>
*Applies a filter to groups created by GROUP BY, allowing you to specify conditions for the aggregated results. It is used after GROUP BY to filter group results based on an aggregate function.*<br>

**EXAMPLE**<br>
Filter regions with sales above a threshold.

In [ ]:
SELECT
    sales_region,
    SUM(amount) AS total_sales
FROM
    sales_data
GROUP BY
    sales_region
HAVING
    SUM(amount) > 10000;

GROUP BY aggregates data into groups, while HAVING filters those groups based on aggregate conditions, both playing pivotal roles in SQL for data summarization and analysis.

## **PIVOT AND UNPIVOT OPERATIONS** <br>

### **PIVOT**<br>
*Converts unique values from one column into multiple columns, allowing for aggregation of another column's values. This operation is useful for transforming row-based data into a more readable, column-based format for analysis.*<br>

**EXAMPLE**<br>
Sales data aggregation by year.

In [ ]:
SELECT product, '2018', '2019', '2020'
FROM sales_data
PIVOT(
  SUM(total_sales)
  FOR year IN ('2018', '2019', '2020')
) AS PivotTable;

### **UNPIVOT**<br>
*Transforms columns into rows, essentially the inverse of PIVOT. This is particularly useful for normalizing data that was previously pivoted or inherently columnar, making it more suitable for row-oriented processing.*<br>

**EXAMPLE**<br>
Normalizing yearly sales data.

In [ ]:
SELECT product, year, total_sales
FROM yearly_sales
UNPIVOT(
  total_sales FOR year IN ('2018', '2019', '2020')
) AS UnpivotTable;

## **PERFORMANCE OPTIMIZATION AND INDEXING** <br>

*Performance optimization and indexing are crucial for enhancing the efficiency of SQL queries, especially when dealing with large datasets. These techniques reduce execution time and resource usage, ensuring fast data retrieval and processing.*<br><br>
**Key Concepts:**
- **Indexing:** Creating indexes on tables to speed up query processing. Common types include B-tree and hash indexes.
- **Query Optimization:** Writing queries in a way that minimizes computation and I/O operations. Includes selecting only necessary columns, using joins efficiently, and filtering data early.
- **Execution Plans:** Analyzing execution plans to understand how a query is executed and identifying bottlenecks.
- **Partitioning:** Dividing tables into smaller, more manageable pieces, improving query performance by limiting the number of rows to scan.
- **Caching:** Utilizing caching mechanisms to store frequently accessed data in memory, reducing the need to access the disk.

**EXAMPLE**<br>
To enhance the performance of a query that retrieves customer information based on transactions:
1. **Index Customer IDs:** Create an index on the customer_id column in the transactions table to speed up searches.
2. **Optimize the Query:** Select only the columns necessary for the report and use WHERE clauses to filter rows early in the query execution.

In [ ]:
CREATE INDEX idx_customer_id ON transactions (customer_id);

SELECT
    name,
    email
FROM
    customers
    JOIN transactions ON customers.customer_id = transactions.customer_id
WHERE
    transactions.purchase_date BETWEEN '2023-01-01' AND '2023-01-31';